In [118]:
import pandas as pd
import numpy as np

import pandascharm as pc
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings("ignore")

In [46]:
df = pd.read_excel('Machine Learning Data.xlsx')

In [47]:
# Download
# pip install pandas-charm

In [48]:
sequence=df["Sequence"]
organisms=df["Organism Name"]

count=0
ofile = open("seqs.fasta", "w")
for i in range(len(sequence)):
    if str(sequence[i]) != 'nan':
        ofile.write(">" + str(organisms[i]) + "\n" + str(sequence[i]) + "\n")
        count=count+1
ofile.close()

In [49]:
from Bio.Align.Applications import ClustalwCommandline
in_file="seqs.fasta"
clustalw_cline=ClustalwCommandline("clustalw2", infile=in_file)

print(clustalw_cline)

clustalw2 -infile=seqs.fasta


In [31]:
# import os
# clustalw_exe =  r"/Users/ryanjang123/Documents/clustalw/clustalw2.exe"
# clustalw_cline=ClustalwCommandline(clustalw_exe, infile='seqs.fasta')

In [44]:
# assert os.path.isfile(clustalw_exe), 'Clustal W executable is missing'
# stdout = clustalw_cline()
# print(clustalw_cline)

check = clustalw_cline.align
print(check)

False


In [50]:
from Bio import AlignIO
ClustalAlign = AlignIO.read('seqs.aln', 'clustal')
summary_align = AlignInfo.SummaryInfo(ClustalAlign )

In [54]:
dframe = pc.from_bioalignment(ClustalAlign).transpose()

In [56]:
dframe

,0,1,2,3,4,5,6,7,8,9,...,1287,1288,1289,1290,1291,1292,1293,1294,1295,1296
AgrobacteriumTumefaciens5A,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
AgrobacteriumTumefaciens5AW127,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
AgrobacteriumTumefaciens5AL178,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
AgrobacteriumTumefaciens5AL178,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
AgrobacteriumTumefaciens5AC174,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CellulomonasFimiATCC484B,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
CellulomonasFimiNag3A,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
ThermobifidaFuscaYX,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
CellulomonasFimiHex20A,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [51]:
data=df[['Wild/Mutant', 'Kingdom', 'pH Optimum', 'Temperature Optimum',
       't1/2 (min)', 'kd (min-1)', 'pNP-Glc Km (mM)', 'pNP-Glc kcat (1/s)',
       'pNP-Glc kcat/Km (1/smM)', 'Cellobiose Km (mM)',
       'Cellobiose kcat (1/s)', 'Cellobiose kcat/Km (1/smM)', 'Cellobiose Ki',
       'pNP-Glc Ki', 'MW (kDa)']]

In [108]:
general_info=data[['Wild/Mutant','Kingdom','pH Optimum','Temperature Optimum','MW (kDa)']]
stability_df=data[['t1/2 (min)','kd (min-1)']]
kinetics_pnp=data[['pNP-Glc Km (mM)', 'pNP-Glc kcat (1/s)', 'pNP-Glc kcat/Km (1/smM)', 'pNP-Glc Ki']]
kinetics_cellobiose=data[['Kingdom','Cellobiose Km (mM)', 'Cellobiose kcat (1/s)', 'Cellobiose kcat/Km (1/smM)', 'Cellobiose Ki']]


#removal of outliers

def removeoutliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_out.dropna()

In [109]:
kcatPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc kcat (1/s)']])
kcatPnpGlc

,pNP-Glc kcat (1/s)
0,5.7000
1,1.7000
2,28.8000
3,277.9150
4,95.0562
...,...
189,0.1000
190,0.7000
191,13.9000
192,0.0000


In [110]:
kcatPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc kcat (1/s)']])
KmPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc Km (mM)']])
kcatKmPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc kcat/Km (1/smM)']])
KiPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc Ki']])
stability=removeoutliers(stability_df[['t1/2 (min)']])

kcat_index = list(kcatPnpGlc.index)
Km_index = list(KmPnpGlc.index)
kcatKm_index = list(kcatKmPnpGlc.index)
Ki_index = list(KiPnpGlc.index)
stability_index = list(stability.index)

# kcatKmCellobiose=kinetics_cellobiose[['Cellobiose kcat/Km (1/smM)']]
# kcatCellobiose=kinetics_cellobiose[[ 'Cellobiose kcat (1/s)']]
# KmCellobiose=kinetics_cellobiose[['Cellobiose Km (mM)']]

In [111]:
# Machine Learning Tools

def LR(X_train, y_train, X_test):
    from sklearn.linear_model import LinearRegression
    lm = LinearRegression()
    lm.fit(X_train,y_train)
    lin_pred = lm.predict(X_test)
    pred_training= lm.predict(X_train)
    
    return lin_pred , pred_training

#Ordinary Least Squares Regression
def OLS(X_train, y_train, X_test):
    import statsmodels.api as sm
    from scipy import stats
    X2 = sm.add_constant(X_train)
    est = sm.OLS(y_train,X2)
    est2 = est.fit()
    print(est2.summary())
    coefficients=est2.params[1:]
    intercept=est2.params[0]
    OLS_pred=(coefficients*X_test).sum(axis = 1, skipna = True) + intercept
    
    return OLS_pred

#Lasso regression

def LASSO(X_train, y_train, X_test):
    param_grid={'alpha':np.arange(0,1.1,0.05).tolist()}
    lasso_reg=Lasso()
    grid_search=GridSearchCV(lasso_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    lasso_pred=grid_search.predict(X_test)
    pred_training= grid_search.predict(X_train)
    return lasso_pred, pred_training

#Partial Least Square Regression

def PLS(X_train, y_train, X_test):
    param_grid={'n_components':np.arange(1,10,1).tolist()}
    pls = PLSRegression()
    grid_search=GridSearchCV(pls, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    pls.fit(scale(X_train), y_train)
    PLS_pred=pls.predict(scale(X_test))
    pred_training= pls.predict(X_train)
    
    return PLS_pred , pred_training

#Random Forest

def RF(X_train, y_train, X_test):
    param_grid={"n_estimators": [100] , "max_features": [5], 'random_state':[20]}
    rf = RandomForestRegressor()
    grid_search=GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    RF_pred = grid_search.predict(X_test)
    pred_training= grid_search.predict(X_train)
    return RF_pred, pred_training

#Decision Tree

def DT(X_train, y_train, X_test):
    tree_reg=DecisionTreeRegressor()
    tree_reg.fit(X_train, y_train)
    DT_pred=tree_reg.predict(X_test)
    pred_training= tree_reg.predict(X_train)
    return DT_pred, pred_training

#SVM

def SVM(X_train, y_train, X_test):
    model = SVR(kernel='poly', degree=2, C=100, epsilon=0.5)
    model.fit(X_train,y_train)
    SVM_pred = model.predict(X_test)
    pred_training= model.predict(X_train)
    return SVM_pred, pred_training

#Neural Network Regression

def NNR(X_train, y_train, X_test):
    regr = MLPRegressor(random_state=101, max_iter=100).fit(X_train, y_train)
    NNR_pred=regr.predict(X_test)
    pred_training= regr.predict(X_train)
    return NNR_pred, pred_training

#Elastic Net

def EN(X_train, y_train, X_test):
    ENmodel = ElasticNet(alpha=1.0, l1_ratio=0.5)
    ENmodel.fit(X_train,y_train)
    EN_pred = ENmodel.predict(X_test)
    pred_training= ENmodel.predict(X_train)
    return EN_pred, pred_training

In [112]:
# 1 pNp OneHot Encoder
X = df['Sequence'].iloc[kcat_index].dropna()
rem_kcat_index = list(X.index)
y = kinetics_pnp['pNP-Glc kcat (1/s)'].iloc[rem_kcat_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [113]:
encoded.transform(temp_seq).toarray().shape

(126, 122)

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [119]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [120]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [121]:
results

[['Linear Regression',
  0.32172412094836556,
  67.95258101246554,
  0.9999826515438375,
  -0.008523119962413173],
 ['LASSO Regression',
  0.3217280227463722,
  67.63783610598486,
  0.9999826511230385,
  0.000797861668396016],
 ['Partial Least Square',
  68.95646258597098,
  67.91518883104655,
  0.20302713311172838,
  -0.007413507837428224],
 ['Decision Tree Regression',
  0.32172412094836556,
  76.57057986738646,
  0.9999826515438375,
  -0.2805537630677364],
 ['Random Forest Regression',
  25.598637683285926,
  67.69262408989586,
  0.890168404088632,
  -0.0008215408968632598],
 ['Support Vector Machine Regression',
  34.682485455420284,
  67.85860831075992,
  0.7983890979208332,
  -0.00573564349226241],
 ['Neural Network Regression',
  86.75560545738229,
  75.82143601243753,
  -0.26150413158993957,
  -0.2556192219446505],
 ['Elastic Network Regression',
  76.03060363835026,
  67.94098517718615,
  0.031118944455735487,
  -0.00817894848353129]]

In [ ]:
# 2 KmpNp OneHot Encoder
X = df['Sequence'].iloc[Km_index].dropna()
rem_km_index = list(X.index)
y = kinetics_pnp['pNP-Glc Km (mM)'].iloc[rem_km_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()